# ストリーミング

## 学習目標

* ストリーミングの仕組みを理解する
* ストリームイベントを扱えるようになる


`anthropic` SDK を import して、クライアントをセットアップしましょう：


In [1]:
from dotenv import load_dotenv
from anthropic import Anthropic

#load environment variable
load_dotenv()

#automatically looks for an "ANTHROPIC_API_KEY" environment variable
client = Anthropic()

ここまで私たちは、次のような書き方で Claude にメッセージを送ってきました：


response = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "Write me an essay about macaws and clay licks in the Amazon",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=800,
    temperature=0,
)
print("We have a response back!")
print("========================")
print(response.content[0].text)

In [2]:
response = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "Write me an essay about macaws and clay licks in the Amazon",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=800,
    temperature=0,
)
print("We have a response back!")
print("========================")
print(response.content[0].text)

We have a response back!
Here is an essay about macaws and clay licks in the Amazon:

Macaws and Clay Licks in the Amazon

Deep within the lush, verdant rainforests of the Amazon basin, a remarkable natural phenomenon takes place. Flocks of brilliantly colored macaws, their vibrant plumage shimmering in the dappled sunlight, gather at special sites known as clay licks. These unique geological formations play a vital role in the lives of these magnificent birds, providing them with essential nutrients and minerals that are crucial for their survival and well-being.

Macaws, with their striking red, blue, yellow, and green feathers, are among the most iconic and captivating inhabitants of the Amazon. These large parrots are known for their impressive size, their raucous calls, and their remarkable intelligence. They are highly social creatures, often seen soaring through the canopy in noisy, chattering groups. But it is at the clay licks where these birds truly come into their own, gathe

この方法でも問題なく動きますが、このやり方では **すべての内容が生成し終わってから** はじめて API から内容が返ってくる点に注意してください。上のセルをもう一度実行すると、全体の応答が一括で表示されるまで何も出力されないはずです。

多くの場面ではこれで十分ですが、ユーザーが「応答が全部生成されるまで待たされる」アプリを作っている場合、体験が悪くなりやすいです。

**そこでストリーミングです！**

ストリーミングを使うと、モデルが生成した内容を「生成され次第」受け取れるため、全体が完成するのを待つ必要がありません。`claude.ai` のようなアプリはこの仕組みで、生成された内容がブラウザへ順次送られて表示されます：

![claude_streaming](images/claude_streaming.gif)


## ストリームを扱う

API からストリーミング応答を得るには、`client.messages.create` に `stream=True` を渡すだけです。ここまでは簡単です。少しややこしいのは、その後にストリーミング応答をどう扱い、流れてくるデータをどう処理するかです。


In [3]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a 3 word sentence, without a preamble.  Just give me 3 words",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=100,
    temperature=0,
    stream=True,
)

`stream` 変数の中身を見てみましょう。


In [4]:
stream

見た目にはあまり情報がありません！この stream オブジェクト単体では、ほとんど何もしてくれません。stream はジェネレータで、API から受け取った server-sent events (SSE) を 1 つずつ yield します。

つまり、こちら側で反復（iterate）して、各 SSE を処理するコードを書く必要があります。もう「完成した 1 つの塊」としてデータが返ってくるのではありません。では実際に stream を回してみましょう：


In [5]:
for event in stream:
    print(event)

RawMessageStartEvent(message=Message(id='msg_01W8XFY58p4frrKQQca8X8yj', content=[], model='claude-3-haiku-20240307', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=30, output_tokens=2, server_tool_use=None, service_tier='standard')), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='Cats', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' me', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='ow lou', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='dly.', type='text_delta'), index=0, t

ご覧の通り、API からは多くの SSE（イベント）が届きます。これらの意味をもう少し詳しく見ていきます。次はイベントを色分けして説明した図です：




![streaming_output](images/streaming_output.png)


各ストリームは、次の順序でイベントが流れてきます：

* **MessageStartEvent** - content が空の Message
* **複数の content block**（各ブロックは次を含む）
  * **ContentBlockStartEvent**
  * 1 個以上の **ContentBlockDeltaEvent**
  * **ContentBlockStopEvent**
* 最終メッセージのトップレベルの変化を示す **MessageDeltaEvent**（1個以上）
* 最後に **MessageStopEvent**

上の例では content block は 1 つだけでした。次の図は、その 1 ブロックに関連するイベント全体を示します：

![content_block_streaming](images/content_block_streaming.png)


私たちが本当に欲しい「モデルが生成した本文」は、`ContentBlockDeltaEvent` から届きます（`type` が `"content_block_delta"`）。実際のテキストは `delta` の中の `text` にあります。生成されたテキストだけを表示してみましょう：


In [6]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a 3 word sentence, without a preamble.  Just give me 3 words",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=100,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "content_block_delta":
        print(event.delta.text)

Cats
 me
ow lou
dly.


テキスト自体は出せていますが、出力が分断されて読みづらいです。Python の `print()` でストリーミングテキストを表示するときは、次の 2 つの引数が便利です：

* `end=""`: `print()` はデフォルトで末尾に改行（`
`）を付けますが、`end=""` にすると改行しません。次の `print()` が同じ行に続いて表示されます。
* `flush=True`: バッファを待たずに即時出力します。ストリーミングで「リアルタイムに表示」したいときに有効です。

これを反映してみましょう：


In [7]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a 3 word sentence, without a preamble.  Just give me 3 words",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=100,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "content_block_delta":
        print(event.delta.text, flush=True, end="")

Cats meow loudly.

これくらい短いテキストだと、ストリーミングの効果が分かりにくいかもしれません。もう少し長いものを生成させてみましょう：


In [9]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            # "content": "How do large language models work?",
            "content": "large language modelsはどのように動作しますか?",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "content_block_delta":
        print(event.delta.text, flush=True, end="")

大規模言語モデル(LLM)の動作原理は以下のようになります:

1. データ収集:大量のテキストデータ(書籍、ウェブページ、会話記録など)を収集する。

2. データ前処理:収集したデータを機械学習モデルに適した形式に変換する。単語の分割、文章の正規化など。

3. モデルの構築:深層学習のアーキテクチャ(トランスフォーマーなど)を使ってモデルを構築する。

4. 学習:大量のテキストデータを使ってモデルを訓練する。単語の共起関係や文脈を学習する。

5. 推論:新しい入力に対して、学習したパターンに基づいて文章を生成したり、質問に答えたりする。

6. 微調整:特定のタスクや分野に合わせて、事前学習済みのモデルを微調整することもできる。

このように、大規模なデータセットを活用し、深層学習のアーキテクチャを活用することで、LLMは人間の言語処理能力に迫る性能を発揮できるようになっています。ただし、バイアスの問題や安全性の課題など、まだ解決すべき課題も残されています。

まだなら、上のセルを実行してみてください。テキストが少しずつ（増分で）表示されるはずです。


ここまで見た通り、`ContentBlockDeltaEvent` にモデルが生成したテキストが入っています。ただし、他のイベントも重要です。例えばトークン使用量を知りたい場合、見るべき場所は 2 つあります：

* `MessageStartEvent` に入力（プロンプト）のトークン使用量が入る
* `MessageDeltaEvent` に出力トークン数が入る

![streaming_tokens](images/streaming_tokens.png)

では、上のコードを更新して、入力トークン数と出力トークン数を表示してみましょう：


In [10]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "large language modelsはどのように動作しますか?",
            # "content": "How do large language models work?",
        }
    ],
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "message_start":
        input_tokens = event.message.usage.input_tokens
        print("MESSAGE START EVENT", flush=True)
        print(f"Input tokens used: {input_tokens}", flush=True)
        print("========================")
    elif event.type == "content_block_delta":
        print(event.delta.text, flush=True, end="")
    elif event.type == "message_delta":
        output_tokens = event.usage.output_tokens
        print("\n========================", flush=True)
        print("MESSAGE DELTA EVENT", flush=True)
        print(f"Output tokens used: {output_tokens}", flush=True)
        

MESSAGE START EVENT
Input tokens used: 21
大規模言語モデル(LLM)の動作原理は以下のようになります:

1. データ収集:大量のテキストデータ(書籍、ウェブページ、会話記録など)を収集する。

2. データ前処理:収集したデータを機械学習モデルに適した形式に変換する。単語の正規化、文章の分割など。

3. モデルの構築:深層学習のアーキテクチャ(トランスフォーマーなど)を使ってモデルを構築する。

4. 学習:大量のテキストデータを使ってモデルを訓練する。単語の共起関係や文脈を学習し、言語の理解と生成を行えるようになる。

5. 推論:学習済みのモデルを使って、入力された文章に対して適切な出力を生成する。単語の予測、文章の生成、質問への回答など。

6. 微調整:特定のタスクや分野に合わせて、モデルを追加の学習(fine-tuning)することで性能を向上させる。

このように、大規模な学習データと強力な深層学習アーキテクチャを組み合わせることで、LLMは人間の言語理解や生成に迫る性能を発揮できるようになっています。ただし、バイアスの問題や安全性の課題など、まだ解決すべき課題も残されています。
MESSAGE DELTA EVENT
Output tokens used: 428


### その他のストリーミングイベントタイプ

ストリームを扱っていると、他にも次のようなイベントに遭遇することがあります：

* **Ping イベント** - ストリームには任意個の ping が含まれ得ます。
* **Error イベント** - ストリーム内にエラーイベントが混ざることがあります。例えば高負荷時には `overloaded_error` が返ることがあり、非ストリーミングの場合の HTTP 529 に相当します。

エラーイベントの例：

```
event: error
data: {"type": "error", "error": {"type": "overloaded_error", "message": "Overloaded"}}
```


## Time to first token (TTFT)

ストリーミングを使う最大の理由は、**TTFT（Time to first token）** を改善することです。これは、あなた（またはユーザー）がモデル生成の最初の内容を受け取るまでの時間を指します。

ストリーミングが TTFT に与える影響をデモしてみましょう。

まずは非ストリーミングです。長い文章を生成させますが、`max_tokens=500` で打ち切ります：


In [ ]:
import time
def measure_non_streaming_ttft():
    start_time = time.time()

    response = client.messages.create(
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": "アメリカの独立戦争の歴史を長文で説明してください",
                # "content": "Write mme a long essay explaining the history of the American Revolution",
            }
        ],
        temperature=0,
        model="claude-3-haiku-20240307",
    )

    response_time = time.time() - start_time

    print(f"Time to receive first token: {response_time:.3f} seconds")
    print(f"Time to recieve complete response: {response_time:.3f} seconds")
    print(f"Total tokens generated: {response.usage.output_tokens}")
    
    print(response.content[0].text)

In [12]:
measure_non_streaming_ttft()

Time to receive first token: 4.205 seconds
Time to recieve complete response: 4.205 seconds
Total tokens generated: 500
アメリカの独立戦争は、1775年から1783年にかけて起こった重要な出来事でした。この戦争は、イギリス植民地であったアメリカ13州がイギリス王国からの独立を勝ち取るために行われたものです。

戦争の発端は、1763年のフランス・インド戦争の終結後、イギリス政府がアメリカ植民地に対して課税を強化したことにありました。植民地住民は、「課税には代表がなければならない」という原則に基づき、これに反発しました。1773年のボストン茶会事件では、植民地住民がイギリス東インド会社の茶税に抗議して茶船を襲撃する事件が起こりました。

これに対してイギリス政府は強硬な対応をとり、植民地に軍隊を派遣しました。1775年4月、マサチューセッツ州レキシントンとコンコードで、イギリス軍と植民地民兵の間で最初の武力衝突が起こりました。これがいわゆる「レキシントン・コンコードの戦い」です。この戦いを機に、植民地住民は本格的な独立戦争を始めることになりました。

独立戦争では、ジョージ・ワシントンを最高司令官とする植民地軍が、イギリス軍と激しい戦闘を繰り広げました。1776年7月4日、13の植民地が「独立宣言」を発表し、正式にイギリスからの独立を宣言しました。その後、フランスやスペインなどが植民地側に加わり、1


次に、同じことをストリーミングでやってみます：


In [13]:
def measure_streaming_ttft():
    start_time = time.time()

    stream = client.messages.create(
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": "アメリカの独立戦争の歴史を長文で説明してください",
                # "content": "Write mme a long essay explaining the history of the American Revolution",
            }
        ],
        temperature=0,
        model="claude-3-haiku-20240307",
        stream=True
    )
    have_received_first_token = False
    for event in stream:
        if event.type == "content_block_delta":
            if not have_received_first_token:
                ttft = time.time() - start_time
                have_received_first_token = True
            print(event.delta.text, flush=True, end="")
        elif event.type == "message_delta":
            output_tokens = event.usage.output_tokens
            total_time = time.time() - start_time

    print(f"\nTime to receive first token: {ttft:.3f} seconds", flush=True)
    print(f"Time to recieve complete response: {total_time:.3f} seconds", flush=True)
    print(f"Total tokens generated: {output_tokens}", flush=True)
    


In [14]:
measure_streaming_ttft()

アメリカの独立戦争は、1775年から1783年にかけて起こった重要な出来事でした。この戦争は、イギリス植民地であったアメリカ13州がイギリス王国からの独立を勝ち取るために行われたものです。

戦争の発端は、1763年のフランス・インド戦争の終結後、イギリス政府がアメリカ植民地に対して課税を強化したことにありました。植民地住民は、「課税には代表がなければならない」という原則に基づき、これらの税金に反対しました。1773年のボストン茶会事件では、植民地住民がイギリス東インド会社の茶船に乗り込み、茶葉を海に投げ捨てるという事件が起こりました。

これに対してイギリス政府は強硬な対応をとり、ボストン港の封鎖や植民地議会の解散など、植民地住民の反発を招きました。1775年4月、マサチューセッツ州レキシントンとコンコードで、イギリス軍と植民地民兵の間で最初の武力衝突が起こりました。これがいわゆる「レキシントン・コンコードの戦い」で、独立戦争の始まりとなりました。

その後、1776年7月4日に独立宣言が採択され、アメリカ合衆国の誕生が宣言されました。独立戦争では、ジョージ・ワシントン将軍率いる植民地軍とイギリス軍の間で激しい戦闘が繰り広げられました。1777年のサラトガの戦い
Time to receive first token: 0.429 seconds
Time to recieve complete response: 3.832 seconds
Total tokens generated: 500


結果を比較してみましょう。

* **ストリーミングなし**
  * **最初のトークンを受け取るまで:** 4.194 秒
  * **応答全体を受け取るまで:** 4.194 秒
  * **生成トークン数:** 500
* **ストリーミングあり**
  * **最初のトークンを受け取るまで:** 0.492 秒
  * **応答全体を受け取るまで:** 4.274 秒
  * **生成トークン数:** 500

見ての通り、TTFT に大きな差があります。このデモは 500 トークンで、しかも最速モデルの Haiku を使っています。もし Opus で 1000 トークン生成する例にすると、数値はさらに大きく変わります。


In [ ]:
def compare_ttft():
    def measure_streaming_ttft():
        start_time = time.time()

        stream = client.messages.create(
            max_tokens=1000,
            messages=[
                {
                    "role": "user",
                    "content": "アメリカの独立戦争の歴史を非常に文字数の多い長文で説明してください",
                    # "content": "Write mme a very very long essay explaining the history of the American Revolution",
                }
            ],
            temperature=0,
            model="claude-opus-4-1-20250805",
            # model="claude-3-opus-20240229",
            stream=True
        )
        have_received_first_token = False
        for event in stream:
            if event.type == "content_block_delta":
                if not have_received_first_token:
                    ttft = time.time() - start_time
                    have_received_first_token = True
            elif event.type == "message_delta":
                output_tokens = event.usage.output_tokens
                total_time = time.time() - start_time
        return (ttft, output_tokens)
    
    def measure_non_streaming_ttft():
        start_time = time.time()

        response = client.messages.create(
            max_tokens=1000,
            messages=[
                {
                    "role": "user",
                    "content": "Write mme a very very long essay explaining the history of the American Revolution",
                }
            ],
            temperature=0,
            model="claude-opus-4-1-20250805"
            # model="claude-3-opus-20240229"
        )
        ttft = time.time() - start_time
        return (ttft, response.usage.output_tokens)
    
    streaming_ttft, streaming_tokens = measure_streaming_ttft()
    non_streaming_ttft, non_streaming_tokens = measure_non_streaming_ttft()

    print("OPUS STREAMING")
    print(f"Time to first token: {streaming_ttft}")
    print(f"Tokens generated: {streaming_tokens}")
    print("#########################################################")
    print("OPUS NON STREAMING")
    print(f"Time to first token: {non_streaming_ttft}")
    print(f"Tokens generated: {non_streaming_tokens}")

        

In [20]:
# DO NOT RUN THIS! It takes over a minute to run and generates around 2000 tokens with Opus! 
compare_ttft()

OPUS STREAMING
Time to first token: 1.5231685638427734
Tokens generated: 1000
#########################################################
OPUS NON STREAMING
Time to first token: 32.76705503463745
Tokens generated: 1000


Opus でより長い文章を生成すると、ストリーミングの TTFT 改善はさらに分かりやすくなります。非ストリーミングでは最初のトークンを受け取るまで 47 秒かかりましたが、ストリーミングでは 1.8 秒で受け取れました。

**注:** ストリーミングはモデルの総生成時間を魔法のように短縮するわけではありません。最初のデータは早く届きますが、リクエスト開始から最後のトークン受信までの総時間が劇的に短くなるわけではありません。


## ストリーミング用ヘルパー


Python SDK には、ストリーミングを扱うための便利機能がいくつかあります。`client.messages.create(stream=True)` の代わりに `client.messages.stream()` を使うと、便利なヘルパーメソッドにアクセスできます。`client.messages.stream()` は `MessageStreamManager`（コンテキストマネージャ）を返し、そこからイベントを emit しつつメッセージを蓄積する `MessageStream` を扱えます。

次の例では `client.messages.stream` を使い、`stream.text_stream` で「テキスト delta だけ」を簡単に取り出して表示します。イベント種別を毎回チェックする必要がありません。

さらに `get_final_message` という便利メソッドもあり、ストリームを最後まで読み終えた後に「最終的な蓄積済みメッセージ」を返してくれます。ストリーミング表示をしつつ、最終的に完成した全文も必要なときに便利です。

次の例は、届いたテキストを順次表示しつつ、完了後に最終メッセージも表示します：


In [4]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()

async def streaming_with_helpers():
    async with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "蘭についてのsonnetを作成してください",
                # "content": "Write me sonnet about orchids",
            }
        ],
        model="claude-opus-4-1-20250805",
    ) as stream:
        async for text in stream.text_stream:
            print(text, end="", flush=True)

    final_message = await stream.get_final_message()
    print("\n\nSTREAMING IS DONE.  HERE IS THE FINAL ACCUMULATED MESSAGE: ")
    print(final_message.to_json())

await streaming_with_helpers()

# 蘭に寄せて

紫の花弁（はなびら）優雅に開きて
朝露に濡れし姿の気高さよ
温室の中に静かに息づきて
異国の香りを運ぶ風のごと

胡蝶蘭の白き羽ばたきは
天上の使いか夢の化身か
カトレアの赤き情熱は燃えて
深き森より届く生命（いのち）かな

シンビジウムの黄金なる輝き
デンドロビウムの可憐な姿
千の種類に千の物語

永遠（とわ）に咲き継ぐ美の極致
人の心を魅了して止まず
蘭よ、気品の花よ、永遠（とこしえ）に

STREAMING IS DONE.  HERE IS THE FINAL ACCUMULATED MESSAGE: 
{
  "id": "msg_01KdWBprhgBjcGv6PQqb5HRP",
  "content": [
    {
      "citations": null,
      "text": "# 蘭に寄せて\n\n紫の花弁（はなびら）優雅に開きて\n朝露に濡れし姿の気高さよ\n温室の中に静かに息づきて\n異国の香りを運ぶ風のごと\n\n胡蝶蘭の白き羽ばたきは\n天上の使いか夢の化身か\nカトレアの赤き情熱は燃えて\n深き森より届く生命（いのち）かな\n\nシンビジウムの黄金なる輝き\nデンドロビウムの可憐な姿\n千の種類に千の物語\n\n永遠（とわ）に咲き継ぐ美の極致\n人の心を魅了して止まず\n蘭よ、気品の花よ、永遠（とこしえ）に",
      "type": "text"
    }
  ],
  "model": "claude-opus-4-1-20250805",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation": {
      "ephemeral_1h_input_tokens": 0,
      "ephemeral_5m_input_tokens": 0
    },
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "input_tokens": 21,
    "output_tokens



`client.messages.stream()` を使う場合、ストリームイベントを処理するために `async for event in stream:` を使ってイベントを反復処理できます。

次の例では、モデルに "generate a 5-word poem" と頼み、ストリームからイベントを受け取りながら処理します：

* `event.type == "text"` - テキスト delta イベントの場合、`event.text` に生成されたテキストが含まれます。この例では、生成テキストを緑色で表示します。
* すべてのイベントタイプを `event.type` で確認できます（デバッグ用）。

**注意**: 最新のAnthropic SDK（0.75.0以降）では、`event_handler`パラメータはサポートされていません。代わりに、`async for event in stream:`を使ってイベントを処理してください。


In [ ]:
from anthropic import AsyncAnthropic, AsyncMessageStream

client = AsyncAnthropic()

green = '\033[32m'
reset = '\033[0m'

class MyStream(AsyncMessageStream):
    async def on_text(self, text, snapshot):
        # This runs only on text delta stream messages
        print(green + text + reset, flush=True) #model generated content is printed in green

    async def on_stream_event(self, event):
        # This runs on any stream event
        print("on_event fired:", event.type)

async def streaming_events_demo():
    async with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Generate a 5-word poem",
            }
        ],
        model="claude-3-opus-20240229",
        event_handler=MyStream,
    ) as stream:
        # Get the final accumulated message, after the stream is exhausted
        message = await stream.get_final_message()
        print("accumulated final message: ", message.to_json())

await streaming_events_demo()

現在のSDKでは AsyncMessageStreamクラスをevent_handlerとして渡す 方式は廃止されている。
代わりに async for event in stream: でイベントに応じた処理を記述するスタイル

In [7]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()

green = '\033[32m'
reset = '\033[0m'

async def streaming_events_demo():
    async with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Generate a 5-word poem",
            }
        ],
        model="claude-opus-4-1-20250805",
    ) as stream:
        # ストリームイベントを処理する
        async for event in stream:
            # テキストイベントの場合、緑色で表示
            if event.type == "text":
                print(green + event.text + reset, end="", flush=True)
            # その他のイベントタイプを確認したい場合は、以下をコメントアウト
            # print("on_event fired:", event.type)
        
        # ストリームが終了したら、最終メッセージを取得
        message = await stream.get_final_message()
        print("\naccumulated final message: ", message.to_json())

await streaming_events_demo()

Morning coffee steams, dreams fade.
accumulated final message:  {
  "id": "msg_01AvBVk9wDjUqCxsut2Jhuxd",
  "content": [
    {
      "citations": null,
      "text": "Morning coffee steams, dreams fade.",
      "type": "text"
    }
  ],
  "model": "claude-opus-4-1-20250805",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation": {
      "ephemeral_1h_input_tokens": 0,
      "ephemeral_5m_input_tokens": 0
    },
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "input_tokens": 14,
    "output_tokens": 11,
    "service_tier": "standard"
  }
}


Python SDKでは、`async for event in stream:`を使って、以下のようなイベントタイプを処理できます：

##### `event.type == "message_start"`
メッセージの開始を示すイベントです。`event.message`に初期Messageオブジェクトが含まれます。

##### `event.type == "text"`
テキスト delta イベントです。`event.text`に生成されたテキストチャンクが含まれます。

##### `event.type == "content_block_stop"`
完全な ContentBlock が蓄積されたときに発火します。`event.content_block`にContentBlockオブジェクトが含まれます。

##### `event.type == "message_stop"`
メッセージの終了を示すイベントです。`event.message`に完全なMessageオブジェクトが含まれます。

##### `event.type == "message_delta"`
メッセージの更新を示すイベントです。`event.usage`にトークン使用量などの情報が含まれます。

すべてのイベントタイプは、`async for event in stream:`ループ内で`event.type`をチェックして処理できます。


***

## 演習

ストリーミングを使うシンプルな Claude チャットボットを書いてください。次の GIF が動作イメージです。出力の色分けは完全に任意で、GIF を見やすくするためのものです：

![streaming_chat_exercise](images/streaming_chat_exercise.gif)


### 解答例
上の演習の 1 つの実装例です。できればこのノートブックのセルとしてではなく、単体の Python スクリプトとして実行するのがおすすめです：


In [ ]:
from anthropic import Anthropic

# Initialize the Anthropic client
client = Anthropic()

# ANSI color codes
BLUE = "\033[94m"
GREEN = "\033[92m"
RESET = "\033[0m"

def chat_with_claude():
    print("Welcome to the Claude Chatbot!")
    print("Type 'quit' to exit the chat.")
    
    conversation = []
    
    while True:
        user_input = input(f"{BLUE}You: {RESET}")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        conversation.append({"role": "user", "content": user_input})
        
        print(f"{GREEN}Claude: {RESET}", end="", flush=True)
        
        stream = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1000,
            messages=conversation,
            stream=True
        )
        
        assistant_response = ""
        for chunk in stream:
            if chunk.type == "content_block_delta":
                content = chunk.delta.text
                print(f"{GREEN}{content}{RESET}", end="", flush=True)
                assistant_response += content
        
        print()  # New line after the complete response
        
        conversation.append({"role": "assistant", "content": assistant_response})

if __name__ == "__main__":
    chat_with_claude()

Welcome to the Claude Chatbot!
Type 'quit' to exit the chat.


***


## ここから独自の実験
非常に長い文章のストリーミングテスト

In [2]:
stream = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": "アメリカの独立戦争の歴史を非常に文字数の多い長文で説明してください",
            # "content": "How do large language models work?",
        }
    ],
    model="claude-opus-4-1-20250805",
    max_tokens=1000,
    temperature=0,
    stream=True,
)
for event in stream:
    if event.type == "message_start":
        input_tokens = event.message.usage.input_tokens
        print("MESSAGE START EVENT", flush=True)
        print(f"Input tokens used: {input_tokens}", flush=True)
        print("========================")
    elif event.type == "content_block_delta":
        print(event.delta.text, flush=True, end="")
    elif event.type == "message_delta":
        output_tokens = event.usage.output_tokens
        print("\n========================", flush=True)
        print("MESSAGE DELTA EVENT", flush=True)
        print(f"Output tokens used: {output_tokens}", flush=True)
        

MESSAGE START EVENT
Input tokens used: 38
# アメリカ独立戦争：自由と独立への長き道のり

## 第1章：独立への胎動 - 植民地時代の背景と緊張の高まり

### 植民地アメリカの形成と発展

17世紀初頭から18世紀中葉にかけて、北アメリカ大陸の東海岸には13のイギリス植民地が形成されていました。1607年のバージニア植民地建設を皮切りに、マサチューセッツ、ニューヨーク、ペンシルベニア、メリーランド、コネチカット、ロードアイランド、デラウェア、ノースカロライナ、サウスカロライナ、ニュージャージー、ニューハンプシャー、ジョージアと、それぞれ異なる経緯と目的を持って建設された植民地は、18世紀中頃までに人口約200万人を擁する一大植民地群へと成長していました。

これらの植民地は、地理的条件や建設の経緯により、大きく三つの地域に分類されていました。ニューイングランド植民地（マサチューセッツ、コネチカット、ロードアイランド、ニューハンプシャー）は、ピューリタンを中心とした宗教的動機による入植者が多く、小規模農業、漁業、造船業、商業が発達していました。中部植民地（ニューヨーク、ペンシルベニア、ニュージャージー、デラウェア）は、多様な民族と宗教が混在し、穀物生産と商業の中心地として栄えていました。南部植民地（バージニア、メリーランド、ノースカロライナ、サウスカロライナ、ジョージア）は、タバコ、米、藍などのプランテーション農業が主体で、奴隷制度に依存した経済構造を持っていました。

### 七年戦争とその影響

1756年から1763年にかけて戦われた七年戦争（アメリカではフレンチ・インディアン戦争と呼ばれる）は、イギリスとフランスの間で北アメリカの覇権を巡って争われた戦争でした。この戦争でイギリスは勝利を収め、1763年のパリ条約によってフランスから広大な北アメリカ領土を獲得しました。しかし、この勝利は同時に巨額の戦費による財政危機をもたらし、イギリス政府は植民地への課税強化によってこの負債を解消しようと考えるようになりました。

さらに、1763年10月7日に発布された「1763年宣言」は、アパラチア山脈以西への植民地人の入植を禁止し、この地域を先住民居留地として保護することを定めました。これは先住民との紛争を避け、統治コストを

ストリーミングを非同期ではなく実行するとどうなるか

In [5]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()

def streaming_with_helpers():
    with client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "蘭についてのsonnetを作成してください",
                # "content": "Write me sonnet about orchids",
            }
        ],
        model="claude-opus-4-1-20250805",
    ) as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)

    final_message = stream.get_final_message()
    print("\n\nSTREAMING IS DONE.  HERE IS THE FINAL ACCUMULATED MESSAGE: ")
    print(final_message.to_json())

streaming_with_helpers()

/tmp/ipykernel_46498/2716704181.py:6: RuntimeWarning: coroutine 'AsyncAPIClient.post' was never awaited
  with client.messages.stream(


TypeError: 'AsyncMessageStreamManager' object does not support the context manager protocol